In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR


data = pd.read_csv(r"newdata_coronary_health_step3.csv",error_bad_lines=False)

data = data[['年龄', '齿列_sum', '平均收缩压', 'WHtR',
             '心电图label', '性别', '空腹血糖', '症状_sum',
             '血小板', '体温','血尿素氮',
             'label','local']]

data_train = data[data['local']!=5.0]
data_External = data[data['local']==5.0]
data = data_train
data.drop(['local',],axis = 1,inplace = True)

data.rename(columns=({'年龄':'Age','齿列_sum':'Dentition', '平均收缩压':'MSP','WHtR':'WHtR',
                    '心电图label':'ECG','空腹血糖':'FBG','症状_sum':'Symptom',
                       '血小板': 'Thrombocyte','体温':'BT', 
                      '血尿素氮':'BUN',  }), inplace = True) 


mdata = data[data['gender']==1.0]
fedata = data[data['gender']==2.0]

mdata_External = data_External[data_External['gender']==1.0]
fedata_External = data_External[data_External['gender']==2.0]

mdata_External.rename(columns=({'年龄':'Age','齿列_sum':'Dentition', '平均收缩压':'MSP','WHtR':'WHtR',
                    '心电图label':'ECG','空腹血糖':'FBG','症状_sum':'Symptom',
                       '血小板': 'Thrombocyte','体温':'BT', 
                      '血尿素氮':'BUN',  }), inplace = True) 

# 模型，粗略跑一下查看结果
X = mdata.iloc[:,mdata.columns != "label"]
y = mdata.iloc[:,mdata.columns == "label"]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

from sklearn.model_selection import train_test_split
X = pd.DataFrame(X)
y = pd.DataFrame(y)

X_train, X_vali, Y_train, Y_vali = train_test_split(X,y,test_size=0.3,random_state=420)
model_data = pd.concat([Y_train, X_train], axis=1)


model_data.index = range(model_data.shape[0])



vali_data = pd.concat([Y_vali, X_vali], axis=1)
vali_data.index = range(vali_data.shape[0])


import matplotlib.pyplot as plt
import scipy
def graphforbestbin(name,DF, X, Y, n,q=20,graph=True):

    
    DF = DF[[X,Y]].copy()

    DF["qcut"],bins = pd.qcut(DF[X], retbins=True, q=q,duplicates="drop")
    coount_y0 = DF.loc[DF[Y]==0].groupby(by="qcut").count()[Y]
    coount_y1 = DF.loc[DF[Y]==1].groupby(by="qcut").count()[Y]
    num_bins = [*zip(bins,bins[1:],coount_y0,coount_y1)]

    for i in range(q):
        if 0 in num_bins[0][2:]:
            num_bins[0:2] = [(
                num_bins[0][0],
                num_bins[1][1],
                num_bins[0][2]+num_bins[1][2],
                num_bins[0][3]+num_bins[1][3])]
            continue

        for i in range(len(num_bins)):
            if 0 in num_bins[i][2:]:
                num_bins[i-1:i+1] = [(
                    num_bins[i-1][0],
                    num_bins[i][1],
                    num_bins[i-1][2]+num_bins[i][2],
                    num_bins[i-1][3]+num_bins[i][3])]
                break
        else:
            break

    def get_woe(num_bins):
        columns = ["min","max","count_0","count_1"]
        df = pd.DataFrame(num_bins,columns=columns)
        df["total"] = df.count_0 + df.count_1
        df["percentage"] = df.total / df.total.sum()
        df["bad_rate"] = df.count_1 / df.total
        df["good%"] = df.count_0/df.count_0.sum()
        df["bad%"] = df.count_1/df.count_1.sum()
        df["woe"] = np.log(df["good%"] / df["bad%"])
        return df

    def get_iv(df):
        rate = df["good%"] - df["bad%"]
        iv = np.sum(rate * df.woe)
        return iv

    IV = []
    axisx = []
    while len(num_bins) > n:
        pvs = []
        for i in range(len(num_bins)-1):
            x1 = num_bins[i][2:]
            x2 = num_bins[i+1][2:]
            pv = scipy.stats.chi2_contingency([x1,x2])[1]
            pvs.append(pv)

        i = pvs.index(max(pvs))
        num_bins[i:i+2] = [(
            num_bins[i][0],
            num_bins[i+1][1],
            num_bins[i][2]+num_bins[i+1][2],
            num_bins[i][3]+num_bins[i+1][3])]

        bins_df = pd.DataFrame(get_woe(num_bins))
        axisx.append(len(num_bins))
        IV.append(get_iv(bins_df))
        
    if graph:
        plt.figure(figsize=(10,7), dpi= 500)
        bwith = 1.5
        TK = plt.gca()
        TK.spines['bottom'].set_linewidth(bwith)
        TK.spines['left'].set_linewidth(bwith)
        TK.spines['top'].set_linewidth(bwith)
        TK.spines['right'].set_linewidth(bwith)
        plt.plot(axisx,IV,label=str(name),linewidth = 3.0)
        font2 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 25,
        'style':'italic'
        }
        font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size' : 16,
        }
        plt.xlabel('number of box',font2)
        plt.ylabel(str(name)+"_IV",font2)

        l1 = plt.legend(loc="lower right",fontsize = 16,prop=font1)
        plt.xticks(axisx)
        plt.xlabel("number of box")
        plt.ylabel(str(name)+"_IV")
        plt.show()

    return bins_df

model_data.columns

column=[ 'Age', 'MSP', 'WHtR','FBG','Thrombocyte','BT', 'BUN', ]#d2

for i in column[0:]:
    print(i)
    graphforbestbin(i,model_data,i,"label",n=2,q=20)

import numpy as np
import pandas as pd
import scipy
import scipy.stats as st

def auto_bin(DF, X, Y, n, iv=True, detail=False,q=20):


    DF = DF[[X,Y]].copy()


    DF["qcut"],bins = pd.qcut(DF[X], retbins=True, q=q, duplicates="drop")

    coount_y0 = DF.loc[DF[Y]==0].groupby(by="qcut")[Y].count()
    coount_y1 = DF.loc[DF[Y]==1].groupby(by="qcut")[Y].count()

    num_bins = [*zip(bins,bins[1:],coount_y0,coount_y1)]


    def get_woe(num_bins):

        columns = ["min","max","count_0","count_1"]
        df = pd.DataFrame(num_bins,columns=columns)

        df["total"] = df.count_0 + df.count_1
        df["percentage"] = df.total / df.total.sum()
        df["bad_rate"] = df.count_1 / df.total
        df["woe"] = np.log((df.count_0/df.count_0.sum()) /
                           (df.count_1/df.count_1.sum()))
        return df

    def get_iv(bins_df):
        rate = ((bins_df.count_0/bins_df.count_0.sum()) -
                (bins_df.count_1/bins_df.count_1.sum()))
        IV = np.sum(rate * bins_df.woe)
        return IV


    for i in range(20): 

        if 0 in num_bins[0][2:]:
            num_bins[0:2] = [(
                num_bins[0][0],
                num_bins[1][1],
                num_bins[0][2]+num_bins[1][2],
                num_bins[0][3]+num_bins[1][3])]
            continue


        for i in range(len(num_bins)):
            if 0 in num_bins[i][2:]:
                num_bins[i-1:i+1] = [(
                    num_bins[i-1][0],
                    num_bins[i][1],
                    num_bins[i-1][2]+num_bins[i][2],
                    num_bins[i-1][3]+num_bins[i][3])]
                break

        else:
            break


    while len(num_bins) > n:

        pvs = []
        for i in range(len(num_bins)-1):
            x1 = num_bins[i][2:]
            x2 = num_bins[i+1][2:]

            pv = st.chi2_contingency([x1,x2])[1]

            pvs.append(pv)


        i = pvs.index(max(pvs))
        num_bins[i:i+2] = [(
            num_bins[i][0],
            num_bins[i+1][1],
            num_bins[i][2]+num_bins[i+1][2],
            num_bins[i][3]+num_bins[i+1][3])]


        bins_df = get_woe(num_bins)
        if iv:
            print(f"{X} 分{len(num_bins):2}组 IV 值: ",get_iv(bins_df))
        if detail:
            print(bins_df)

    return get_woe(num_bins) #, get_iv(bins_df)

auto_col_bins = {'FBG':5,
                'Age':6,
                'MSP':5,
                'WHtR':5,
                 'Thrombocyte':5,
                 'BT':3,
                 'BUN':4,
                }


hand_bins = {"Dentition":[0.0,1.0]
            ,"ECG":[1.0,2.0]
             ,"Symptom":[0.0,1.0]
            }

hand_bins = {k:[-np.inf,*v[:-1],np.inf] for k,v in hand_bins.items()}

bins_of_col = {}
 

 
for col in auto_col_bins:
    bins_df = graphforbestbin(DF = model_data
                             ,X = col
                             ,Y = "label"
                             ,name=col
                             ,q=20
                            ,n = auto_col_bins[col]
                             ,graph=False
                             )
    bins_list = sorted(set(bins_df["min"]).union(bins_df["max"]))

    bins_list[0],bins_list[-1] = -np.inf,np.inf
    bins_of_col[col] = bins_list
    

  
bins_of_col.update(hand_bins)
 
bins_of_col

def get_woe(df,col,y,bins):
    df = df[[col,y]].copy()
    df["cut"] = pd.cut(df[col],bins,duplicates = 'drop')
    bins_df = df.groupby("cut")[y].value_counts().unstack()
    woe = bins_df["woe"] = np.log((bins_df[0]/bins_df[0].sum())/(bins_df[1]/bins_df[1].sum()))
    return woe

woeall = {}
for col in bins_of_col:
    woeall[col] = get_woe(model_data,col,"label",bins_of_col[col])
    
woeall

model_data.head()


model_woe = pd.DataFrame(index=model_data.index)
 

for col in bins_of_col:
    model_woe[col] = pd.cut(model_data[col],bins_of_col[col],duplicates='drop').map(woeall[col])
    

model_woe["label"] = model_data["label"]

model_woe.head()

mdata_External['gender'].value_counts()
mdata_External.drop(['gender',],axis = 1,inplace = True)

mdata_External.head()


model_woe_External = pd.DataFrame(index=mdata_External.index)
 
 

for col in bins_of_col:
    model_woe_External[col] = pd.cut(mdata_External[col],bins_of_col[col],duplicates='drop').map(woeall[col])


model_woe_External["label"] = mdata_External["label"]


vali_X_External = model_woe_External.iloc[:,:-1]
vali_y_External = model_woe_External.iloc[:,-1]

vali_X_External.info()

vali_woe = pd.DataFrame(index=vali_data.index)
 
for col in bins_of_col:
    vali_woe[col] = pd.cut(vali_data[col],bins_of_col[col],duplicates='drop').map(woeall[col])
vali_woe["label"] = vali_data["label"]
 
vali_X = vali_woe.iloc[:,:-1]
vali_y = vali_woe.iloc[:,-1]

vali_X

from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score as accuracy, roc_auc_score as auc
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score,recall_score, f1_score,roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight
 
lr = LR(class_weight= {0:1,1:16},penalty="l2",solver="liblinear",C=0.5).fit(vali_X,vali_y)

lr.score(vali_X,vali_y)
lr_result = lr.predict(vali_X)


sw = compute_sample_weight(class_weight=None,y=vali_y)
lr_cm = confusion_matrix(vali_y, lr_result, sample_weight=sw).ravel()

print("\tAUC:{}".format(auc(vali_y,lr.predict_proba(vali_X)[:,1])))

print("\tAccuracy:{}".format(accuracy(vali_y,lr_result)))
print("\tRecall:{}".format(recall_score(vali_y,lr_result)))
print("\tf1:{}".format(f1_score(vali_y, lr_result)))
print("\tprecision:{}".format(precision_score(vali_y, lr_result)))

print(lr_cm)

lr_result_External = lr.predict(vali_X_External)
sw = compute_sample_weight(class_weight=None,y=vali_y_External)
lr_cm_External = confusion_matrix(vali_y_External, lr_result_External, sample_weight=sw).ravel()

print("\tAUC:{}".format(auc(vali_y_External,lr.predict_proba(vali_X_External)[:,1])))

print("\tAccuracy:{}".format(accuracy(vali_y_External,lr_result_External)))
print("\tRecall:{}".format(recall_score(vali_y_External,lr_result_External)))
print("\tf1:{}".format(f1_score(vali_y_External, lr_result_External)))
print("\tprecision:{}".format(precision_score(vali_y_External, lr_result_External)))
# print("roc_auc %f" % (roc_auc))
print(lr_cm_External)

lr.get_params() 

large = 28; med = 20; small = 12
params = {'axes.titlesize': med, 
          'legend.fontsize': 28, 
          'legend.loc': 'lower right',
          #'figure.figsize': (16, 10), 
          'axes.labelsize': 28, 
          'xtick.labelsize': 28,
          'ytick.labelsize': 28,
          'figure.titlesize': 28} 
plt.rcParams.update(params) 



from sklearn.metrics import roc_curve
FPR, recall, thresholds = roc_curve(vali_y,lr.predict_proba(vali_X)[:,1],pos_label=1)
FPR
recall
thresholds 
from sklearn.metrics import roc_auc_score as AUC
area = AUC(vali_y,lr.predict_proba(vali_X)[:,1])

maxindex = (recall - FPR).tolist().index(max(recall - FPR))


plt.figure(figsize=(10,7), dpi= 500)


bwith = 3
TK = plt.gca()
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)



plt.plot(FPR, recall,color='red',linewidth = 6.0,label='ROC curve (area =  0.8671)' % area)


plt.plot([0, 1], [0, 1], color='black', linestyle='--')
bwith = 1.5 

plt.scatter(FPR[maxindex],recall[maxindex],c="black",s=150)
plt.text(FPR[maxindex]+0.04,recall[maxindex]-0.01,'(best threshold:'+str('%.2f'%thresholds[maxindex])+')',fontsize=25)
print(area)
print(thresholds[maxindex])
print(FPR[maxindex])
print(recall[maxindex])

plt.xlim([-0.02,1.02])
plt.ylim([-0.02,1.02])
font2 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':35,
    'style':'italic'
}
font1 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':25
}
plt.xlabel('1-Specificity',font2)
plt.ylabel('Sensitivily',font2)
l1 = plt.legend(loc = "lower right",fontsize = 25,prop=font1)


# 外部验证ROC
plt.rcParams['font.sans-serif']=['Simhei'] 
plt.rcParams['axes.unicode_minus']=False
from sklearn.metrics import roc_curve
FPR, recall, thresholds = roc_curve(vali_y_External,lr.predict_proba(vali_X_External)[:,1],pos_label=1)
FPR
recall
thresholds 
from sklearn.metrics import roc_auc_score as AUC
area = AUC(vali_y_External,lr.predict_proba(vali_X_External)[:,1])

maxindex = (recall - FPR).tolist().index(max(recall - FPR))


plt.figure(figsize=(10,7), dpi= 500)


bwith = 3
TK = plt.gca()
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)



plt.plot(FPR, recall,color='red',linewidth = 6.0,label='ROC curve (area =  0.824)' % area)



plt.plot([0, 1], [0, 1], color='black', linestyle='--')
bwith = 1.5 

plt.scatter(FPR[maxindex],recall[maxindex],c="black",s=150)
plt.text(FPR[maxindex]+0.04,recall[maxindex]-0.01,'(best threshold:'+str('%.2f'%thresholds[maxindex])+')',fontsize=25)
print(area)
print(thresholds[maxindex])
print(FPR[maxindex])
print(recall[maxindex])

plt.xlim([-0.02,1.02])
plt.ylim([-0.02,1.02])
font2 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':35,
    'style':'italic'
}
font1 = {
    'family':'Times New Roman',
    'weight':'normal',
    'size':25
}
plt.xlabel('1-Specificity',font2)
plt.ylabel('Sensitivily',font2)
l1 = plt.legend(loc = "lower right",fontsize = 25,prop=font1)



import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Simhei'] 
plt.rcParams['axes.unicode_minus']=False
file = "./chdscorecardfemale.csv"
X.drop(['gender',],axis = 1,inplace = True)

base_score = (48.94 - 8.094*lr.intercept_)
with open(file,"w") as fdata:
    base_score = round(base_score[0],1)
    mdata.write("base_score,{}\n".format(base_score,2))
for i,col in enumerate(X.columns):
    score = round((((woeall[col] * (-B*lr.coef_[0][i]))))/3.95,2)
    score.name = "Score"
    score.index.name = col
    score.to_csv(file,header=True,mode="a")



